# Tutoriel Stable Baselines3 - Premiers pas


Stable-Baselines3 : https://github.com/DLR-RM/stable-baselines3

Documentation : https://stable-baselines3.readthedocs.io/en/master/

RL Baselines3 zoo : https://github.com/DLR-RM/rl-baselines3-zoo


[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) est un framework d’entraînement pour l’Apprentissage par Renforcement (RL), basé sur Stable Baselines3.

Il fournit des scripts pour entraîner des agents, les évaluer, réaliser des recherches d’hyperparamètres, tracer des courbes de résultats et enregistrer des vidéos.

Source:  https://github.com/araffin/rl-tutorial-jnrr19

## Introduction

Dans ce notebook, vous allez apprendre les bases d’utilisation de la librairie Stable Baselines3 : comment créer un modèle RL, l’entraîner et l’évaluer. Comme toutes les algorithmes partagent la même interface, nous verrons qu’il est très simple de passer d’un algorithme à un autre.

## Installer les dépendances et Stable Baselines3 avec Pip

La liste complète des dépendances est disponible dans le [README](https://github.com/DLR-RM/stable-baselines3).

Pour installer :
```
pip install stable-baselines3[extra]
```

---
**Rappels sur l’Apprentissage par Renforcement (AR)**  
- Un agent interagit avec un environnement au fil de pas de temps.  
- À chaque pas, il reçoit un `state` (observation), choisit une `action` et obtient une `reward`.  
- Le but est de maximiser la somme des récompenses.  
- Stable-Baselines3 fournit un ensemble d’algorithmes pour entraîner cet agent sur divers environnements.

*Astuce* : Pour ceux qui découvrent Gym, explorez rapidement `env.action_space` et `env.observation_space` pour comprendre les dimensions et les types d’actions.


In [1]:
# Sous Windows, on ne fait pas d'apt-get.
# %apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # Pour la visualisation sous Linux
%pip install "stable-baselines3[extra]>=2.0.0a4" moviepy

  Using cached gymnasium-1.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached opencv_python-4.13.0.92-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pygame-2.6.1-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached rich-14.3.3-py3-none-any.whl.metadata (18 kB)
  Using cached ale_py-0.11.2-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached markdown-3.10.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached filelock-3.24.3-py3-none-any.whl.metadata (2.0 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached stable_base

  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import stable_baselines3

print(f"{stable_baselines3.__version__=}")

stable_baselines3.__version__='2.8.0a2'


## Imports

Stable-Baselines3 fonctionne avec des environnements qui suivent l’interface [gym](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html).
Vous pouvez trouver une liste d’environnements disponibles [ici](https://gym.openai.com/envs/#classic_control).

Il est aussi recommandé de regarder le [code source](https://github.com/openai/gym) pour en savoir plus sur l’espace d’observation et d’action de chaque environnement, car gym ne fournit pas de documentation très détaillée.
Tous les algorithmes ne sont pas compatibles avec tous les espaces d’action. Vous trouverez plus d’informations dans ce [tableau récapitulatif](https://stable-baselines.readthedocs.io/en/master/guide/algos.html).

In [3]:
import gymnasium as gym
import numpy as np

print(f"{gym.__version__=}")

gym.__version__='1.2.3'


La première chose dont vous avez besoin est d’importer la classe de l’algorithme de RL que vous souhaitez utiliser. Consultez la documentation pour savoir quel algorithme utiliser dans quel contexte.

PPO est un algorithme on-policy, ce qui signifie que les données utilisées pour la mise à jour des réseaux proviennent de la politique courante. À l’inverse, un algo off-policy comme DQN peut réutiliser des données issues de politiques antérieures.

In [4]:
from stable_baselines3 import PPO

Ensuite, vous pouvez importer la classe de politique (policy) qui servira à créer les réseaux (pour la fonction de politique et la fonction de valeur). Ce n’est pas obligatoire : vous pouvez directement utiliser des chaînes de caractères lors de la création du modèle, par exemple :
```PPO('MlpPolicy', env)``` au lieu de ```PPO(MlpPolicy, env)```.

Notez que certains algorithmes comme `SAC` ont leur propre `MlpPolicy`, donc l’utilisation de la chaîne de caractères est généralement recommandée.

In [5]:
from stable_baselines3.ppo import MlpPolicy

## Créer l’environnement Gym et instancier l’agent

Dans cet exemple, nous allons utiliser l’environnement CartPole, un problème classique de contrôle.

« Un poteau est attaché par un joint non-actionné à un chariot, qui se déplace le long d’un rail sans frottement. Le système est contrôlé en appliquant une force de +1 ou -1 sur le chariot. Le pendule commence à la verticale, et l’objectif est de l’empêcher de tomber. Une récompense de +1 est accordée à chaque pas de temps pendant lequel le poteau reste en position verticale. »

Environnement CartPole : [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)

Les environnements vectorisés (vecenv) permettent de faciliter l’entraînement en parallèle. Ici, nous utilisons un seul processus, donc `DummyVecEnv`.

Nous choisissons `MlpPolicy` car l’entrée de CartPole est un vecteur de caractéristiques (et non une image).

Le type d’action (discrète/continue) sera automatiquement déduit de l’espace d’action de l’environnement.

Ici, nous utilisons [Proximal Policy Optimization](https://stable-baselines.readthedocs.io/en/master/modules/ppo2.html), qui est une méthode Actor-Critic : elle utilise une fonction de valeur pour améliorer la descente de gradient de la politique (en réduisant la variance).

PPO combine des idées d’[A2C](https://stable-baselines.readthedocs.io/en/master/modules/a2c.html) (plusieurs workers et bonus d’entropie pour encourager l’exploration) et de [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) (utilisation d’une région de confiance pour stabiliser l’apprentissage et éviter des chutes drastiques de performance).

PPO est un algorithme on-policy : les trajectoires utilisées pour mettre à jour les réseaux doivent être collectées avec la politique la plus récente.
Il est généralement moins échantillonnement-efficace que des algorithmes off-policy comme [DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines.readthedocs.io/en/master/modules/sac.html) ou [TD3](https://stable-baselines.readthedocs.io/en/master/modules/td3.html), mais il est souvent plus rapide en temps d’horloge réel.

In [6]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
model = PPO(MlpPolicy, env, verbose=0)

Nous créons une fonction utilitaire pour évaluer l’agent :

In [7]:
def evaluate(model, num_episodes=100, deterministic=True):
    """
    Évalue un agent RL.
    :param model: (BaseRLModel) l’agent RL
    :param num_episodes: (int) nombre d’épisodes sur lesquels évaluer
    :param deterministic: (bool) si on utilise une politique déterministe
    :return: (float) Récompense moyenne sur les num_episodes derniers épisodes
    """
    # Cette fonction ne fonctionne que pour un seul environnement
    vec_env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = vec_env.reset()
        while not done:
            # _states n’est utile que lorsque l’on utilise des politiques LSTM
            action, _states = model.predict(obs, deterministic=deterministic)
            # Ici, action, rewards et dones sont des tableaux
            # car nous utilisons un environnement vectorisé
            obs, reward, done, info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Récompense moyenne :", mean_episode_reward, "Nombre d’épisodes :", num_episodes)

    return mean_episode_reward

En fait, Stable-Baselines3 fournit déjà un utilitaire similaire :

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

Évaluons l’agent non entraîné ; il devrait agir de façon essentiellement aléatoire.

In [9]:
# Nous utilisons un environnement distinct pour l’évaluation
eval_env = gym.make("CartPole-v1", render_mode="rgb_array")

# Agent aléatoire, avant entraînement
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

C:\Users\jsboi\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:9.29 +/- 0.77


## Entraîner l’agent et l’évaluer

**Hyperparamètres clés**  
- `total_timesteps`: nombre total de pas d’entraînement (interactions avec l’environnement).  
- `learning_rate`: définit la vitesse à laquelle les poids sont mis à jour.  
- `n_steps` (ou équivalent): longueur des trajectoires collectées avant chaque mise à jour, etc.  
- `batch_size`: taille de l’échantillon pour chaque itération d’apprentissage.  

*Tip* : N’hésitez pas à ajuster progressivement `total_timesteps` si la convergence n’est pas satisfaisante.


In [10]:
# Entraînons l’agent pendant 10 000 pas
model.learn(total_timesteps=10_000)

In [11]:
# Évaluons l’agent entraîné
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:449.70 +/- 68.65


Visiblement, l’entraînement s’est bien déroulé : la récompense moyenne a beaucoup augmenté !

### Préparer l’enregistrement vidéo

**Note sur la visualisation**  
- Sous Windows, on n’a pas besoin de créer de display virtuel (`xvfb`).  
- Sur Linux, si vous n’avez pas d’interface graphique, vous devrez lancer un display virtuel pour capturer des frames (`xvfb-run`).  
- Les fonctions ci-dessous utilisent `render_mode=\"rgb_array\"` pour récupérer les images directement.


In [12]:
# Sous Windows, pas besoin de lancer un display virtuel.
# On commente donc la partie suivante (utile surtout sous Linux) :
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

In [13]:
import base64
from pathlib import Path
from IPython import display as ipythondisplay

def show_videos(video_path="", prefix=""):
    """
    Inspiré de : https://github.com/eleurent/highway-env

    :param video_path: (str) chemin vers le dossier contenant les vidéos
    :param prefix: (str) filtre sur le préfixe des noms de fichiers vidéo
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Nous allons enregistrer une vidéo à l’aide de [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder). Vous en apprendrez davantage sur ces wrappers dans le prochain notebook.

In [14]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("CartPole-v1", render_mode="rgb_array")])
    # Commencer la vidéo au pas=0 et enregistrer 500 étapes
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Fermer le recorder vidéo
    eval_env.close()

### Visualiser l’agent entraîné


In [15]:
record_video("CartPole-v1", model, video_length=500, prefix="ppo-cartpole")

Saving video to C:\dev\CoursIA\videos\ppo-cartpole-step-0-to-step-500.mp4
MoviePy - Building video C:\dev\CoursIA\videos\ppo-cartpole-step-0-to-step-500.mp4.
MoviePy - Writing video C:\dev\CoursIA\videos\ppo-cartpole-step-0-to-step-500.mp4



frame_index:   0%|          | 0/500 [00:00<?, ?it/s, now=None]

frame_index:  21%|██        | 105/500 [00:00<00:00, 1040.10it/s, now=None]

frame_index:  48%|████▊     | 239/500 [00:00<00:00, 1215.73it/s, now=None]

frame_index:  75%|███████▌  | 377/500 [00:00<00:00, 1290.07it/s, now=None]

MoviePy - Done !
MoviePy - video ready C:\dev\CoursIA\videos\ppo-cartpole-step-0-to-step-500.mp4


In [16]:
show_videos("videos", prefix="ppo")

## Bonus : entraîner un modèle RL en une seule ligne

La classe de politique utilisée sera déduite automatiquement et l’environnement sera créé automatiquement également. Cela fonctionne parce que les deux sont [enregistrés](https://stable-baselines.readthedocs.io/en/master/guide/quickstart.html).

In [17]:
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.9     |
|    ep_rew_mean     | 22.9     |
| time/              |          |
|    fps             | 1372     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


## Conclusion

Dans ce notebook, nous avons vu :
- comment définir et entraîner un modèle RL à l’aide de Stable Baselines3 (en une seule ligne de code) ;)
